In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PATH = '../finanz_vs_news/'

In [3]:
df_reu = pd.read_csv(PATH+'reuters_2008.csv.gz', parse_dates=['Time']).set_index('Time')

In [4]:
df_reu.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 771666 entries, 2008-01-02 04:59:53 to 2008-08-20 04:00:00
Data columns (total 1 columns):
Short_Msg    771666 non-null object
dtypes: object(1)
memory usage: 11.8+ MB


In [5]:
#temporary measure to reduce suspicious dates
df_reu = df_reu.loc['2008',:]

### Read in company ticker names

In [6]:
companies = pd.read_pickle('TickerDic.pickle')

In [7]:
companies.head()

,Ticker,Name,Exchange,Category Name,Country
0,AFL,Aflac Incorporated,NYQ,Accident & Health Insurance,USA
1,AIZ,"Assurant, Inc.",NYQ,Accident & Health Insurance,USA
23,UNM,Unum Group,NYQ,Accident & Health Insurance,USA
52,IPG,"The Interpublic Group of Companies, Inc.",NYQ,Advertising Agencies,USA
57,OMC,Omnicom Group Inc.,NYQ,Advertising Agencies,USA


In [8]:
company_names = companies['Name'].values

### Do bag of words

In [9]:
import helpers

In [17]:
media_attention = helpers.count_sensitive_words(df_reu, company_names)

In [18]:
media_attention

,Aflac_Incorporated_count,"Assurant,_Inc._count",Unum_Group_count,"The_Interpublic_Group_of_Companies,_Inc._count",Omnicom_Group_Inc._count,Northrop_Grumman_Corporation_count,Textron_Inc._count,The_Boeing_Company_count,General_Dynamics_Corporation_count,"L3_Technologies,_Inc._count",...,"Under_Armour,_Inc._count","Coach,_Inc._count","Fossil_Group,_Inc._count","NIKE,_Inc._count","Hasbro,_Inc._count",PACCAR_Inc_count,"Republic_Services,_Inc._count","Stericycle,_Inc._count","Waste_Management,_Inc._count",Sprint_Corporation_count
Time,,,,,,,,,,,,,,,,,,,,,
2008-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-07,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-08,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008-01-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
